<a href="https://colab.research.google.com/github/codedyasai/Python_Intermediate/blob/main/2_9_%EC%B6%95%EA%B5%AC%EA%B2%BD%EA%B8%B0%EA%B2%B0%EA%B3%BC%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국제 축구 경기 분석

- 1872년부터 2018년까지 전 세계에서 개최된 국제 축구 대회 데이터를 분석한다.
- 데이터셋에 6년간 대한민국 축구팀 경기 분석 결과를 분석해봅시다.

In [ ]:
## 필수 라이브러리 로딩
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# Word Cloud(or Tag Cloud, 태그 클라우드)
!pip install wordcloud

In [ ]:
import seaborn as sns
from wordcloud import WordCloud
# 한글 패키지 Konlpy

- 라이브러리 or 모듈 : 파일명.py

In [ ]:
soccer = pd.read_csv('soccer.csv')

# 탐색적 데이터 분석(EDA)

## 데이터셋의 기본정보 확인

In [ ]:
soccer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41540 entries, 0 to 41539
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        41540 non-null  object
 1   home_team   41540 non-null  object
 2   away_team   41540 non-null  object
 3   home_score  41540 non-null  int64 
 4   away_score  41540 non-null  int64 
 5   tournament  41540 non-null  object
 6   city        41540 non-null  object
 7   country     41540 non-null  object
 8   neutral     41540 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 2.6+ MB


In [ ]:
soccer.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [ ]:
# date 컬럼에 데이터타입을 날짜와 시간형식으로 변경
#soccer.astype(['date': 'datetime64[ns]'])
soccer.date = pd.to_datetime(soccer.date)

In [ ]:
soccer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41540 entries, 0 to 41539
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        41540 non-null  datetime64[ns]
 1   home_team   41540 non-null  object        
 2   away_team   41540 non-null  object        
 3   home_score  41540 non-null  int64         
 4   away_score  41540 non-null  int64         
 5   tournament  41540 non-null  object        
 6   city        41540 non-null  object        
 7   country     41540 non-null  object        
 8   neutral     41540 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(5)
memory usage: 2.6+ MB


In [ ]:
# date컬럼을 기준으로 내림차순 정렬
# 정렬 결과를 soccer 데이터셋에 저장
soccer.sort_values(by='date', ascending= False, inplace= True)

## 인사이트 도출
1. 홈경기와 원정경기의 결과를 분석

In [ ]:
# 홈경기와 원정경기를 분석
## 경기가 진행된 중립지역을 나타내는 컬럼의 값이 False에 해당되는 데이터를 대상으로 분석
## 홈에서 이루어진 경기의 결과를 승과 패를 담는 파생변수를 생성

In [ ]:
# 승리팀을 담는 컬럼 soccer['winner']

In [ ]:
## 경기가 진행된 중립지역(neutral)을 나타내는 컬럼의 값이 False에 해당되는 데이터를 대상으로 분석
df = soccer[soccer['neutral']== False]

In [ ]:
soccer['neutral'] == False

41539    True
41517    True
41515    True
41514    True
41513    True
         ... 
4        True
3        True
2        True
1        True
0        True
Name: neutral, Length: 41540, dtype: bool

In [ ]:
# 경기결과: 2-1 식으로 표현
def final_score(df):
    max_s = max(df.home_score, df.away_score)
    min_s = min(df.home_score, df.away_score)
    return '{}-{}'.format(max_s, min_s)

In [ ]:
soccer['result'] = soccer.apply(final_score, axis= 1)
soccer.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,result
41539,2019-11-19,Lebanon,North Korea,0,0,FIFA World Cup qualification,Beirut,Lebanon,False,0-0
41517,2019-11-19,U.S. Virgin Islands,Saint Martin,1,2,CONCACAF Nations League,Upper Bethlehem,U.S. Virgin Islands,False,2-1
41515,2019-11-19,Saint Lucia,Montserrat,0,1,CONCACAF Nations League,Gros Islet,Saint Lucia,False,1-0
41514,2019-11-19,El Salvador,Dominican Republic,2,0,CONCACAF Nations League,San Salvador,El Salvador,False,2-0
41513,2019-11-19,Mexico,Bermuda,2,1,CONCACAF Nations League,Mexico City,Mexico,False,2-1


In [ ]:
# 원정팀이 방문한 경기에서 홈팀이 획득한 총점수
soccer.groupby('away_team')['home_score'].count()

away_team
Abkhazia          10
Afghanistan       71
Albania          157
Alderney          12
Algeria          213
                ... 
Yugoslavia       290
Zambia           418
Zanzibar         147
Zimbabwe         266
Åland Islands     19
Name: home_score, Length: 306, dtype: int64

In [ ]:
# 국가명을 인덱스로
away_ind = pd.DataFrame(soccer.groupby('away_team')['home_score'].count().index)

In [ ]:
# 원정팀에 대해 홈팀이 획득한 점수
away_val = pd.DataFrame(soccer.groupby('away_team')['home_score'].count().values, columns=['Value'])

In [ ]:
home_total = pd.concat([away_ind, away_val], axis=1)
home_total.head()

,away_team,Value
0,Abkhazia,10
1,Afghanistan,71
2,Albania,157
3,Alderney,12
4,Algeria,213


In [ ]:
import pandas as pd

# 데이터 프레임 생성
data = {'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
df = pd.DataFrame(data)

# rolling 함수를 사용하여 이동 평균 계산
window_size = 3  # 이동 평균을 계산할 윈도우 크기
rolling_mean = df['value'].rolling(window=window_size).mean()

print(rolling_mean)

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
Name: value, dtype: float64
